# Initiate Bert Server

In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
import tensorflow as tf
print(tf.__version__)

1.15.2


In [ ]:
!pip install bert-serving-client
!pip install -U bert-serving-server[http]

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 358kB 12.1MB/s 
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7411 sha256=593be6fa16c7be963c67820811c227593e51a84b0b25683d87c523df08cd2cb0
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
  Created wheel for flask-compress: filename=Flask_Compress-1.5.0-cp36-none-any.whl size=5272 sha256=b649f311f7e47bbf6268d3d87662c4300f9b41d896523a482b657406b1b5bd57
  Stored in directory: /root/.cache/pip/wheels/f7/e9/e4/5afc286be7c87461375e33152558415dfeb0c8f5af3b50e742
Successfully built GPUtil flask-compress


In [ ]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!unzip uncased_L-12_H-768_A-12.zip

--2020-09-08 07:01:15--  https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 142.250.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 407727028 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 388.84M   145MB/s    in 2.7s    

2020-09-08 07:01:19 (145 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [407727028/407727028]

Archive:  uncased_L-12_H-768_A-12.zip
   creating: uncased_L-12_H-768_A-12/
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  


In [ ]:
!nohup bert-serving-start -pooling_strategy=NONE -max_seq_len=200 -show_tokens_to_client -model_dir=./uncased_L-12_H-768_A-12 >out.file2>&1&

nohup: redirecting stderr to stdout


# Load Corpus

In [4]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
corpus = pd.read_csv('./drive/My Drive/IoT_Corpus/agg_corpora.csv')
corpus = corpus.dropna()
corpus['Comment'] = corpus['Comment'].fillna('none')
corpus['Title'] = corpus['Title'].fillna('none')
corpus = corpus.drop('Unnamed: 0', axis=1)
len(corpus)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


1160

In [5]:
import gensim.corpora as corpora
v = []
for i in list(corpus['Comment'].values):
  sentences = i.split(".")
  temp = []
  for j in sentences:
    temp.extend(j.split())
  v.append(temp)
dic = corpora.Dictionary(v)
dic.filter_extremes(no_below=2)
print(dic)

Dictionary(6743 unique tokens: ['also', 'answer', 'appreciate', 'complete', 'comprehensive']...)


In [6]:
def remove_low_occ(x):
  title = x['Title']
  comments = x['Comment']
  title_r = []
  for i in title.split():
    if i in list(dic.token2id.keys()):
      title_r.append(i)
  comment_r = []
  for j in comments.split('.'):
    sent_r = []
    for k in j.split():
      if k in list(dic.token2id.keys()) and len(k)>1:
        sent_r.append(k)
    comment_r.append(" ".join(sent_r))
  x['Title'] = " ".join(title_r)
  x['Comment'] = ".".join(comment_r)
  return x

In [7]:
corpus = corpus.apply(remove_low_occ, axis=1)

# Import Gaussian LDA Model

In [8]:
!pip install git+git://github.com/jcrudy/choldate.git
!pip install gaussianlda
from gaussianlda import GaussianLDAAliasTrainer, GaussianLDATrainer

  Cloning git://github.com/jcrudy/choldate.git to /tmp/pip-req-build-6nvq_nhx
  Running command git clone -q git://github.com/jcrudy/choldate.git /tmp/pip-req-build-6nvq_nhx
  Created wheel for choldate: filename=choldate-0.1.0-cp36-cp36m-linux_x86_64.whl size=59525 sha256=fa101114f4540f542f48614c32a1e2ed11b1da65e23ac02413be300406c47a4d
  Stored in directory: /tmp/pip-ephem-wheel-cache-61z97kqc/wheels/46/56/c6/befdb2ca536ea7da32af9ce0938cf548a518404abe7b1365bf
Successfully built choldate
     |████████████████████████████████| 71kB 2.3MB/s 
  Created wheel for progressbar: filename=progressbar-2.5-cp36-none-any.whl size=12076 sha256=b38df0ea6ca19ec3d890488856ea9e16b366eba996d28e362412cb6e49b172d5
  Stored in directory: /root/.cache/pip/wheels/c0/e9/6b/ea01090205e285175842339aa3b491adeb4015206cda272ff0
Successfully built progressbar


# Util functions

In [9]:
def select_topics(topics):
  out = []
  for word in [topics.split()[i] for i in range(0, len(topics.split()), 2)]:
    if not word.startswith('[') and not word.startswith('#') and word not in out:
      out.append(word)
  return out

In [10]:
def sparse_topics(temp):
  
  topics = []
  for i in temp.split('\n'):
    if i.endswith('unused'):
      continue
    topics.append(i.split(":")[1].strip())
  return topics

In [11]:
def topic_model(title, corpus, embeddings, vocab, embedding_type='bert', w2v=None, num_topics=5, alpha=0.31, sample=1, num_words=10):
  trainer = GaussianLDATrainer(corpus, embeddings, vocab, num_tables=num_topics,alpha=alpha,cholesky_decomp=True)
  trainer.sample(sample)
  temp = trainer.format_topics(num_words=10)
  print(temp)
  if embedding_type == 'bert':
    title_embeddings = embed_title(title)
  else:
    _, title_embeddings, _ = word2vec_embed(title, w2v)
  topics = sparse_topics(temp)
  return title_embeddings, topics


# Evaluation

## Topic Coherence

In [25]:
from gensim.models import CoherenceModel
import gensim.corpora as corpora
def coherence_evaluation(topics, texts, id2word):
  # Compute Coherence Score
  # temp = [' '.join(i) for i in comments]
  model = []
  for i in topics:
    model.append([j for j in i])
  dictionary = corpora.Dictionary(texts)
  coherence_model_lda = CoherenceModel(topics=model,texts=texts, dictionary=dictionary, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print("Coherence Score: %f" %coherence_lda)
  return coherence_lda

## Title Cosine Similarity

In [13]:
from scipy.spatial.distance import cosine
def cosine_sim(titles, topics):
  cs = []
  for title in titles:
    similarity = [1 - cosine(title, topics[0])]
    cs.append(np.mean(similarity))
  return np.mean(cs)

## Stability

In [14]:
from scipy.spatial.distance import cosine
def stability(topics):
  mean_topics = np.mean(topics)
  temp = [1-cosine(mean_topics, j) for j in topics]
  return np.mean(temp) * 10

## Variability

In [15]:
import numpy as np
from scipy.stats import variation 
def variability(topics):
  cvs = []
  for i in range(len(topics[0])):
    cvs.append(variation([j[i] for j in topics]))
  return np.std(cvs)

## Evaluation Pipeline

In [16]:
def evalution_pipiline(topics, title_embeddings, vocab, texts, embeddings):
  topic_embeddings = []
  cs = []
  clean_topics = []
  stabilities = []
  if isinstance(texts[0][0],int):
    texts = [[vocab[i] for i in j] for j in texts]
  for num, i in enumerate(topics):
    topic = select_topics(i)
    clean_topics.append(topic)
    print("Topic %d: %s" % (num, (',').join(topic)))
    print(topic)
    # Cosine Similarity
    topic_embedding = [embeddings[vocab.index(i)] for i in topic]
    cs.append(cosine_sim(title_embeddings, topic_embedding))
    # Stability
    stab = stability(topic_embedding)
    stabilities.append(stab)
  ce = coherence_evaluation(clean_topics, texts, vocab)
  return ce, stabilities, cs, clean_topics

# Hyperparametering

In [ ]:
def hyper_parm(title, w2v_corpus, embeddings, vocab, w2v):
  model_results = {'Coherence':[],
               'Alpha':[],
               'Cosine Similarity':[],
               'Stability':[]}
  for a in list(np.arange(0.01, 1, 0.3)):
    title_embeddings, topics = topic_model(title, w2v_corpus, embeddings, vocab, 'w2v', w2v, num_topics=8, sample=5, alpha=a)
    title_embeddings = list(title_embeddings)
    cv, stab, cs, clean_topics = evalution_pipiline(topics, title_embeddings, vocab, w2v_corpus, embeddings)
    model_results['Alpha'].append(a)
    model_results['Coherence'].append(cv)
    model_results['Cosine Similarity'].append(np.mean(cs))
    model_results['Stability'].append(np.mean(stab))
  results = pd.DataFrame(model_results)
  return results

# Word2Vec Embedding

In [17]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [18]:
import numpy as np
from gensim.models import Word2Vec

In [19]:
def word2vec_embed(corpus, w2v):
  vocab = w2v.wv.vocab
  w2v_corpus = [[vocab[str(i)].index for i in j] for j in corpus]
  embeddings = np.asarray([w2v.wv[i] for i in vocab.keys()])
  w2v_vocab = w2v.wv.index2word
  return w2v_corpus, embeddings, w2v_vocab

In [20]:
def train_w2v(texts):
  w2v = Word2Vec(texts, min_count=1)
  return w2v

In [21]:
def drop_dot(tex):
    texts = [text.replace('.', ' ') for text in tex]
    tokenized_corpus = [nltk.word_tokenize(sen) for sen in texts]
    return tokenized_corpus

In [26]:
from scipy.stats import variation
def glda_w2v(comments, w2v, title):
  w2v_corpus, embeddings, vocab = word2vec_embed(comments, w2v)
  title_embeddings, topics = topic_model(title, 
                        w2v_corpus, 
                        embeddings, 
                        vocab, 
                        'w2v', 
                        w2v, 
                        num_topics=8, 
                        sample=10, 
                        alpha=0.31)
  title_embeddings = list(title_embeddings)
  ce, stab, cs, clean_topics = evalution_pipiline(topics, title_embeddings, vocab, w2v_corpus, embeddings)
  return ce, stab, cs, clean_topics

# Run w2v GLDA

In [29]:
from google.colab import files
texts = list(corpus['Comment'].values)
titles = list(corpus['Title'].values)
clean_texts = drop_dot(texts)
train_text = clean_texts
train_text.extend(titles)
w2v = train_w2v(train_text)

In [ ]:
w2v_corpus, embeddings, vocab = word2vec_embed(clean_texts, w2v)
hyp_param = hyper_parm(titles, w2v_corpus, embeddings, vocab, w2v)
hyp_param.sort_values(by=['Coherence','Cosine Similarity', 'Stability'], ascending=False)

In [30]:
ce, stab, cs, topics = glda_w2v(clean_texts, w2v, titles)
performance = pd.DataFrame({'Word2Vec Coherence': ce, 'Word2Vec Cosine Similarity': cs, 'Word2Vec Stability': stab, 'Topic Estimatation': topics})
performance.to_excel("w2v_performance.xlsx", index=False)
files.download('w2v_performance.xlsx')

2020-09-08 13:22:17,052 - GLDA - INFO - Initializing assignments

2020-09-08 13:22:40,575 - GLDA - INFO - Iteration 0

2020-09-08 13:26:21,048 - GLDA - INFO - Iteration 1

2020-09-08 13:30:00,437 - GLDA - INFO - Iteration 2

2020-09-08 13:33:39,713 - GLDA - INFO - Iteration 3

2020-09-08 13:37:19,781 - GLDA - INFO - Iteration 4

2020-09-08 13:40:56,986 - GLDA - INFO - Iteration 5

2020-09-08 13:44:34,216 - GLDA - INFO - Iteration 6

2020-09-08 13:48:14,352 - GLDA - INFO - Iteration 7

2020-09-08 13:51:53,377 - GLDA - INFO - Iteration 8

2020-09-08 13:55:33,165 - GLDA - INFO - Iteration 9



0: transport (8.77e-01) third (5.41e-02) lorawan (5.09e-02) release (1.47e-02) avoid (2.69e-03) ot (6.60e-05) service (3.92e-05) w (3.18e-05) driver (7.04e-06) parameter (3.83e-06)
1: feature (3.14e-01) suggest (1.76e-01) hear (1.50e-01) demonstrate (1.12e-01) apply (7.52e-02) perhaps (3.52e-02) fact (3.39e-02) offer (1.95e-02) benefit (9.10e-03) io (8.86e-03)
2: humanity (4.42e-03) scrap (4.14e-03) msk (4.11e-03) tolerant (4.09e-03) placement (4.03e-03) rasperry (4.00e-03) wireshark (3.99e-03) buffer (3.94e-03) un (3.94e-03) seamlessly (3.92e-03)
3: go (9.99e-01) rely (8.98e-04) might (9.01e-10) dashboard (5.82e-10) loop (3.90e-12) location (9.39e-14) particle (2.71e-17) secure (7.25e-18) connect (1.26e-18) wikipedia (2.96e-19)
4: wd (6.93e-02) migrate (6.92e-02) sensing (6.77e-02) serf (6.75e-02) choosen (6.66e-02) parser (6.50e-02) populate (6.09e-02) aeotec (6.05e-02) dood (6.05e-02) ug (5.96e-02)
5: wall (5.62e-01) youtube (2.98e-01) want (4.55e-02) industrial (4.39e-02) d (2.56e-

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
ce

In [ ]:
cs

In [ ]:
stab

# BERT Word Embedding

In [ ]:
import numpy as np
from bert_serving.client import BertClient
def embed_title(title):
  client = BertClient()
  embeddings = []
  for i in title:
    if i == '':
      continue
    temp = client.encode([i])
    embeddings.append(np.mean(temp[0]))
  return embeddings

In [ ]:
def bert_embed(comment):
  client = BertClient()
  vocab = []
  embeddings = []
  while len(comment.split()) > 198:
    temp = comment.split()[:198]
    comment = ' '.join(comment.split()[198:])
    temp.insert(0, '[CLS]')
    temp.append('[SEP]')
    embedding = client.encode([' '.join(temp)], show_tokens=True)
    vocab.extend(list(embedding[1][1:-1]))
    embeddings.extend(list(embedding[0][1:-1]))
  embedding = client.encode([comment], show_tokens=True)
  vocab.extend(list(embedding[1][0][1:len(comment.split())+1]))
  embeddings.extend(list(embedding[0][0][1:len(comment.split())+1]))
  return vocab, embeddings, vocab

In [ ]:
def split_dot(tex):
  texts = [text.split('.') for text in tex]
  return texts

In [ ]:
def glda_bert(comments):
  bert_corpus = []
  bert_embeddings = []
  bert_vocab = []
  for i in comments:
    doc_corpus = []
    doc_embeddings = []
    doc_vocab = []
    for j in i:
      if j == '' or not j:
        continue
      corpus, embeddings, vocab = bert_embed(j)
      doc_corpus.extend(corpus)
      doc_embeddings.extend(embeddings)
      doc_vocab.extend(vocab)
    bert_corpus.append(doc_corpus)
    bert_embeddings.extend(doc_embeddings)
    bert_vocab.extend(doc_vocab)
  print(len(bert_vocab))
  print(len(bert_embeddings))
  return bert_corpus, np.asarray(bert_embeddings), bert_vocab

In [ ]:
def bert_word2index(bert_corpus, bert_vocab):
  breakpoint = 0
  for i in range(len(bert_corpus)):
    for j in range(len(bert_corpus[i])):
      bert_corpus[i][j] = bert_vocab.index(bert_corpus[i][j])
  return bert_corpus

# Run BERT GLDA

In [ ]:
texts = list(corpus['Comment'].values)
titles = list(corpus['Title'].values)
clean_texts = split_dot(texts)
print("Embedding")
texts, bert_embeddings, bert_vocab = glda_bert(clean_texts)
bert_corpus = bert_word2index(texts, bert_vocab)

Embedding
191219
191219


In [ ]:
print("Topic Modelling")
title_embeddings, topics = topic_model(titles, bert_corpus, bert_embeddings, bert_vocab, num_topics=10, sample=1)

2020-09-05 09:27:55,467 - GLDA - INFO - Initializing assignments


Topic Modelling



2020-09-05 09:40:11,005 - GLDA - INFO - Iteration 0



0: hardware (9.83e-01) ##p (1.12e-02) connectivity (6.15e-03) gateway (4.03e-10) communication (5.90e-11) technology (1.26e-12) data (5.83e-13) ##u (3.98e-14) technology (6.70e-15) data (1.49e-17)
1: sensor (1.00e+00) sensor (1.36e-05) system (8.77e-06) sensor (6.16e-06) carrier (1.61e-06) technology (1.20e-07) connectivity (5.78e-08) chip (5.76e-08) sensor (4.41e-08) protocol (3.12e-08)
2: move (7.47e-01) move (2.53e-01) want (4.18e-06) talk (8.96e-08) base (5.80e-09) stuff (3.29e-09) think (1.42e-09) love (1.33e-09) move (2.35e-10) program (1.49e-10)
3: ##b (1.00e+00) system (2.93e-04) awesome (6.05e-05) industrial (9.94e-06) love (5.63e-07) hand (1.63e-07) jump (2.65e-08) practical (1.88e-08) power (4.10e-09) support (1.90e-09)
4: ##ta (1.00e+00) smart (3.87e-08) ras (1.73e-08) es (4.43e-12) proto (3.91e-12) pc (3.86e-15) drag (2.57e-15) nu (1.08e-15) ##p (9.94e-16) pre (4.24e-16)
5: integrate (1.00e+00) seem (1.44e-04) interesting (9.90e-06) appropriate (5.31e-08) seem (3.61e-10) i

In [ ]:
from google.colab import files
print("Evaluating")
bert_ce, bert_stab, bert_cs, bert_topics = evalution_pipiline(topics, title_embeddings, bert_vocab, texts, bert_embeddings)
performance = pd.DataFrame({'BERT Coherence': bert_ce, 'BERT Cosine Similarity': bert_cs, 'BERT Stability': bert_stab, 'BERT Topic Estimate': bert_topics})
performance.to_excel("bert_performance.xlsx", index=False)
files.download('bert_performance.xlsx')

Evaluating
Topic 0: hardware,connectivity,gateway,communication,technology,data
['hardware', 'connectivity', 'gateway', 'communication', 'technology', 'data']
Topic 1: sensor,system,carrier,technology,connectivity,chip,protocol
['sensor', 'system', 'carrier', 'technology', 'connectivity', 'chip', 'protocol']
Topic 2: move,want,talk,base,stuff,think,love,program
['move', 'want', 'talk', 'base', 'stuff', 'think', 'love', 'program']
Topic 3: system,awesome,industrial,love,hand,jump,practical,power,support
['system', 'awesome', 'industrial', 'love', 'hand', 'jump', 'practical', 'power', 'support']
Topic 4: smart,ras,es,proto,pc,drag,nu,pre
['smart', 'ras', 'es', 'proto', 'pc', 'drag', 'nu', 'pre']
Topic 5: integrate,seem,interesting,appropriate,involve,possibility,want,classify,deploy
['integrate', 'seem', 'interesting', 'appropriate', 'involve', 'possibility', 'want', 'classify', 'deploy']
Topic 6: need,actually,take,think,maybe,already
['need', 'actually', 'take', 'think', 'maybe', 'alre

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
bert_cs

[0.04442734877173534,
 0.04264783181515494,
 0.04233760016441967,
 0.0454645218484615,
 0.04688678192804939,
 0.047137043570742,
 0.044833768473098216,
 0.04784946826866769,
 0.047942797943859077,
 0.04233760016441967]